# Reconnaissance de caractères

### Peut-on à l'aide du deep learning reconnaitre des chiffres écrit à la main ? 

Chargement de la librairie R de deep learning :

In [ ]:
library(keras)

Le Deep Learning peut être vu comme une sous-catégorie des « réseaux de neurones ». En effet,
il s’agit d’un ensemble d’outils et de méthodes d’apprentissage automatique basés sur l’utilisation
« avancée » de réseaux de neurones. On utilise les mots « deep » ou « profond » en référence au
nombre de couches de neurones qui constituent ces réseaux : plus le nombre de couches est grand plus
le réseau est profond et plus il permet de traiter des problèmes complexes. 

<img src="reseau2neurones.PNG" style="width:600px;height:250px"/>

Chargement du jeu de données MNIST :

In [ ]:
mnist <- dataset_mnist()

La base de données MNIST pour Modified ou Mixed National Institute of Standards and Technology, est une base de données de chiffres écrits à la main. C'est un jeu de données très utilisé en deep learning.

La reconnaissance de l'écriture manuscrite est un problème difficile, et un bon test pour les algorithmes d'apprentissage. La base MNIST est devenue un test standard. Elle regroupe 60000 images d'entrainement et 10000 images de test, issues d'une base de données antérieure, appelée simplement NIST. Ce sont des images en noir et blanc, normalisées centrées de 28 pixels de côté. 

<img src="mnist.png" style="width:700px;height:400px"/>

Il y a 784 pixels ce qui représente une image de dimension 28 x 28.

# 1) Création des jeux d'entrainement et de test

Préparons les images MNIST d'entrainement et de test. Ces images seront notées train_x et test_x.

In [ ]:
train_x <- mnist$train$x
test_x <- mnist$test$x

Dimension du jeu d'entrée d'entrainement et de test :

In [ ]:
dim(train_x)
dim(test_x)

Dans le jeu d'entrée d'entrainement, il y a 60000 images de 28x28 pixels. Idem pour le jeu d'entrée de test, sauf qu'il n'y a que 10000 images ! Ces images ne peuvent pas être traitées telles quelles par le réseau de neurones. Il faut :
- les vectoriser : on met les 784 pixels (28x28) en une seule ligne, appelée vecteur (explication dans l'image ci-dessous)
- les normaliser : les pixels n'ont plus des valeurs comprises entre 0 et 255 mais entre 0 et 1. 
Les deux lignes de commandes suivantes font les deux traitements :

<img src="mnist2.PNG" style="width:300px;height:250px"/>

In [ ]:
train_x <- array(train_x, dim = c(dim(train_x)[1], prod(dim(train_x)[-1]))) / 255
test_x <- array(test_x, dim = c(dim(test_x)[1], prod(dim(test_x)[-1]))) / 255

Malgré la vectorisation, on peut toujours afficher les données d'entrée sous forme d'une image à l'aide des commandes matrix() et image().
Affichons la première image du jeu de d'entrée d'entrainement (train_x) :

In [ ]:
m <- matrix(train_x[1,],28,28,byrow=T)
image(m[,ncol(m):1], axes = FALSE,col=gray.colors(255))

Cette image correspond à cette matrice :

In [ ]:
m

Affichage de la huitième image du jeu de donnée d'entrainement (train_x) :

In [ ]:
m <- matrix(train_x[8,],28,28,byrow=T)
image(m[,ncol(m):1], axes = FALSE,col=gray.colors(255))

Préparartion du jeu de données d'entrainement et de test permettant de comparer si les résultats prédits par le réseau de neurones sont corrects ou pas.

In [ ]:
train_y <- mnist$train$y
test_y <- mnist$test$y

Par défaut, les données sont stockées sous forme de nombre entiers, il est préférable de les stocker sous forme binaire pour faire du deep learning. En effet, on a apprend pas à reconnaitre la forme des nombres mais on apprend si le nombre est reconnu dans la bonne case (la case du 0, du 1, du 2, etc).

In [ ]:
train_y_cat<-to_categorical(train_y,10)
test_y_cat<-to_categorical(test_y,10)
train_y <- train_y_cat
test_y <- test_y_cat

Dimension du jeu Y d'entrainement :

In [ ]:
dim(train_y)

Regardons le premier nombre :

In [ ]:
train_y[1,]

Ce nombre est 5 !

Représentons-le graphiquement :

In [ ]:
nbre <- train_y[1,]
names(nbre) <- 0:9 
barplot(nbre)

Que vaut le centième chiffre du jeu de données test ? 

Suprression du jeu de données entier MNITS de la mémoire :

In [ ]:
rm(mnist)

### Récapitulons ce que l'on vient de voir à l'aide du schéma général du deep learning :
<img src="nn2.png" style="width:700px;height:170px"/>

# 2) construction du réseau de neurones

MNIST est un jeu de données conséquent mais facile pour du deep learning ! Nous allons choisir une couche d'entrée avec 784 neurones (ce qui correspond à nos 784 pixels de nos images - même taille que notre vecteur X). 
Une couche de sortie avec 10 neurones correspondant aux 10 chiffres que l'on veut retrouver (même taille que notre vecteur Y). Entre ces deux couches, on met une couche caché composée de 100 neurones. Pour lier les couches entre elles, on utilise des fonctions d'activation comme relu ou sofmax. Ce réseau est schématisé dans la figure suivante :

<img src="reseau1.png" style="width:270px;height:300px"/>

In [ ]:
model <- keras_model_sequential() 
model %>%
layer_dense(units = 784, input_shape = 784) %>%
layer_activation(activation = 'relu') %>%
layer_dense(units = 100) %>%
layer_activation(activation = 'relu') %>%
layer_dense(units = 10) %>%
layer_activation(activation = 'softmax')
summary(model)

Avant d'entrainer le modèle, il faut le compiler. Cette étape consiste à savoir comment on va évaluer la performance du modèle.

In [ ]:
model %>% compile(
loss = 'categorical_crossentropy',
optimizer = 'adam',
metrics = c('accuracy')
)

On peut maintenant entrainer le modèle à l'aide de la fonction fit(). Le modèle va s'entrainer sur 30 epochs avec 1000 images à chaque epoch.

In [ ]:
learning <- model %>% fit(train_x, train_y, epochs = 30, batch_size = 1000, validation_split=0.2)

Regardons ce qu'il s'est passé pendant l'entrainement :

In [ ]:
plot(learning)

In [ ]:
model %>% evaluate(train_x, train_y)

On obtient une précision du modèle (réseau de neurones) supérieure à 97% et très proche de 100% !

# 3) Evaluation du modèle

Evaluation maintenant la performance du modèle sur le jeu de données test (le résultat sera nécessairement inférieur à celui effectué sur le jeu de données d'entrainement).

In [ ]:
model %>% evaluate(test_x, test_y)

On peut maintenant reconnaitre les caractères du jeu de données test : 

In [ ]:
prediction <- model %>% predict(test_x)

Le jeu de données test_y et prédiction sont de la même taille (c'est rassurant !) et ca permet de les comparer : 

In [ ]:
dim(prediction)
dim(test_y)

Regardons la première image du jeu test à déchiffrer :

In [ ]:
m<-matrix(test_x[235,],28,28,byrow=T)
image(m[,ncol(m):1])


Que voit-on ? le chiffre 7. Est-ce bien le chiffre 7 qu'il fallait lire ? 

In [ ]:
nbre <- test_y[1,]
names(nbre) <- 0:9 
barplot(nbre)

Quel chiffre a été prédit par notre réseau de neurones ? 

In [ ]:
nbre <- prediction[1,]
names(nbre) <- 0:9 
barplot(nbre)
prediction[1,]

Le réseau de neurone a bien reconnu le 7 mais est-ce le cas pour tout notre jeu de données ? Regardons pour les 20 premiers chiffres.

In [ ]:
library(ggplot2)
library(repr)
nbre <-  0:9
nbplot <- 500
prevision <- round(prediction)%*%nbre
objectif = test_y%*%nbre
df <- data.frame(index = 1:nbplot,previ = prevision[1:nbplot], objectif[1:nbplot])
df
ggplot(data=df,aes(x=1:nbplot,y=df$previ-df$objectif)) + geom_point() +theme_bw()

Bravo !!! Vous avez réalisé votre premier réseau de neurones !

# 4) Overfitting 

Il se peut parfois que l'entrainement du modèle ne se passe pas bien... Une des possibilités est l'overfitting : le modèle est "trop" entrainé et colle "trop" aux données sans saisir l'évolution moyenne des données :

<img src="overfiting.PNG" style="width:150px;height:130px"/>

Reprendre la procédure de la construction et de l'évaluation du réseau de neurones présenté dans cette page pour construire le réseau de neurones ayant les caractéristiques suivantes :

In [ ]:
# Overfitting 
model <- keras_model_sequential() 
model %>%
layer_dense(units = 784, input_shape = 784) %>%
layer_activation(activation = 'relu') %>%
layer_dense(units = 1000) %>%
layer_activation(activation = 'relu') %>%
layer_dense(units = 1000) %>%
layer_activation(activation = 'relu') %>%
layer_dense(units = 10) %>%
layer_activation(activation = 'softmax')
summary(model)

Au brouillon, réaliser le schéma (avec le ronds rouges, jaunes et gris) représentant ce nouveau réseau de neurones.

Le modèle réussit-il à mieux évaluer les chiffres que le premier modèle ? 

# 5) Underfitting

Un second problème classique peut survenir si on entraine pas assez bien son réseau de neurones, c'est l'underfitting : le réseau est sous-entrainé !

<img src="underfitting.PNG" style="width:150px;height:130px"/>

Reprendre la procédure de la construction et de l'évaluation du réseau de neurones présenté dans cette page pour construire le réseau de neurones ayant les caractéristiques suivantes :

In [ ]:
# Underfitting 
model <- keras_model_sequential() 
model %>%
layer_dense(units = 784, input_shape = 784) %>%
layer_activation(activation = 'relu') %>%
layer_dense(units = 2) %>%
layer_activation(activation = 'relu') %>%
layer_dense(units = 10) %>%
layer_activation(activation = 'softmax')
summary(model)

Au brouillon, réaliser le schéma (avec le ronds rouges, jaunes et gris) représentant ce nouveau réseau de neurones.